In [12]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer


#Feature Selection
from sklearn.feature_selection import SelectKBest, RFE, f_regression, mutual_info_regression

#Classification
from sklearn.svm import SVR

# Regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor


# Cross-Validation
from sklearn.model_selection import GroupKFold

from biopsykit.classification.model_selection import SklearnPipelinePermuter


%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
save_results = True

In [3]:
data_path = Path("../../results/data")
data_path

WindowsPath('../../results/data')

In [4]:
models_path = Path("../../results/models")

In [5]:
input_data = pd.read_csv(data_path.joinpath("train_data_rr_interval_no_outlier_correction.csv"), index_col=[0,1,2,3,4])
input_data

,,,,,rr_interval_ms_estimated,arbol2017-isoelectric-crossings,arbol2017-second-derivative,arbol2017-third-derivative,debski1993-second-derivative,drost2022,forounzafar2018,lozano2007-linear-regression,lozano2007-quadratic-regression,sherwood1990,stern1985
,participant,phase,heartbeat_id_reference,b_point_sample_reference,,,,,,,,,,,
0,GDN0005,HoldingBreath,0,388.0,850.0,438.0,398.0,394.0,452.0,400.0,550.0,412.0,384.0,442.0,388.0
1,GDN0005,HoldingBreath,1,404.0,778.0,340.0,350.0,244.0,388.0,420.0,402.0,404.0,384.0,330.0,402.0
2,GDN0005,HoldingBreath,3,376.0,746.0,382.0,296.0,386.0,366.0,386.0,388.0,366.0,348.0,382.0,374.0
3,GDN0005,HoldingBreath,4,390.0,766.0,394.0,344.0,396.0,376.0,396.0,398.0,372.0,348.0,394.0,388.0
4,GDN0005,HoldingBreath,5,386.0,790.0,398.0,312.0,388.0,418.0,392.0,390.0,378.0,354.0,400.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11133,VP_032,Talk,39,310.0,619.0,335.0,276.0,324.0,300.0,318.0,306.0,305.0,294.0,337.0,306.0
11134,VP_032,Talk,40,322.0,680.0,331.0,298.0,330.0,311.0,322.0,311.0,321.0,303.0,329.0,287.0
11135,VP_032,Talk,41,340.0,719.0,317.0,300.0,348.0,330.0,343.0,309.0,332.0,322.0,311.0,287.0


In [6]:
print(f"Min data value: {input_data.values.min()}\nMax data value: {input_data.values.max()}")

Min data value: -84.0
Max data value: 1534.0


In [7]:
columns=input_data.columns
columns

Index(['rr_interval_ms_estimated', 'arbol2017-isoelectric-crossings',
       'arbol2017-second-derivative', 'arbol2017-third-derivative',
       'debski1993-second-derivative', 'drost2022', 'forounzafar2018',
       'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
       'sherwood1990', 'stern1985'],
      dtype='object')

In [8]:
negative_data = input_data[input_data.values < 0]
negative_data

,,,,,rr_interval_ms_estimated,arbol2017-isoelectric-crossings,arbol2017-second-derivative,arbol2017-third-derivative,debski1993-second-derivative,drost2022,forounzafar2018,lozano2007-linear-regression,lozano2007-quadratic-regression,sherwood1990,stern1985
,participant,phase,heartbeat_id_reference,b_point_sample_reference,,,,,,,,,,,
783,GDN0007,Valsalva,0,448.0,928.0,392.0,394.0,458.0,434.0,458.0,-84.0,440.0,410.0,388.0,448.0
6583,VP_002,Math,22,209.0,456.0,219.0,190.0,-9.0,199.0,217.0,168.0,236.0,234.0,216.0,118.0
6622,VP_002,Math,45,209.0,438.0,224.0,184.0,-15.0,200.0,216.0,183.0,230.0,228.0,223.0,133.0
6624,VP_002,Math,46,206.0,441.0,220.0,183.0,-16.0,197.0,211.0,139.0,229.0,228.0,219.0,139.0
6625,VP_002,Math,47,206.0,442.0,221.0,181.0,-18.0,199.0,214.0,183.0,228.0,227.0,224.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10589,VP_031,Math,34,201.0,432.0,175.0,158.0,-38.0,192.0,214.0,197.0,216.0,217.0,203.0,197.0
10591,VP_031,Math,35,199.0,433.0,218.0,160.0,-34.0,189.0,208.0,191.0,218.0,218.0,219.0,191.0
10614,VP_031,Math,49,197.0,423.0,206.0,154.0,-31.0,182.0,202.0,154.0,212.0,212.0,209.0,107.0


In [9]:
X, y, groups, group_keys = bp.classification.utils.prepare_df_sklearn(data=input_data, label_col="b_point_sample_reference", subject_col="participant", print_summary=True)

Shape of X: (10305, 11); shape of y: (10305,); number of groups: 39, class prevalence: [ 2  1  1  4  1  1  1  1  1  2  1  1  2  1  1  2  3  2  6  3  9  4  8  7
 10  4  5 11  7 13  9 11 14 11 15  9 13 15 17 20 22 20 28 26 28 27 34 35
 43 37 37 39 30 33 46 40 50 47 42 38 23 19 26 34 34 27 33 37 26 40 19 25
 33 23 23 29 30 18 28 25 18 29 17 20 24 24 23 11 18 20 16 23 26 24 21 10
 15 12 16 14 17 20 16 13 17 16 25 16 12 19 21 18 19 19 22 17 22 18 20 24
 24 18 26 21 17 23 13 14 22 21 25 12 21 17 27 14 26 17 20 23 18  9 22 16
 24 11 17 20 25 16 31 20 23 25 25 24 27  9 34 14 35 12 36 21 29 15 32 22
 55 16 48 14 38 27 31 11 52  8 54 19 53 23 50 14 52 18 63 19 56 18 55 24
 47 21 59 15 57 11 71 19 43 18 55 12 72 16 56 13 52 16 63 18 76 12 84  9
 60 15 69  9 58  4 66 16 64  7 63  9 82  9 82  6 73  8 71  9 80 10 89  8
 69  7 68  5 74  5 71  2 61  7 79  5 77  2 82  7 72  8 64  8 61  1 80  2
 76  3 63 73  2 68  3 63  2 85  6 65  2 84  2 78  1 66  2 64 71 73  2 56
  1 69 69  1 59  1 74 49 66  1 66 63 

In [10]:
group_keys

Index(['GDN0005', 'GDN0006', 'GDN0007', 'GDN0008', 'GDN0009', 'GDN0010',
       'GDN0011', 'GDN0012', 'GDN0013', 'GDN0014', 'GDN0016', 'GDN0017',
       'GDN0018', 'GDN0019', 'GDN0020', 'GDN0021', 'GDN0022', 'GDN0023',
       'GDN0024', 'GDN0025', 'GDN0027', 'GDN0028', 'GDN0029', 'GDN0030',
       'VP_001', 'VP_002', 'VP_003', 'VP_004', 'VP_005', 'VP_020', 'VP_022',
       'VP_023', 'VP_026', 'VP_027', 'VP_028', 'VP_029', 'VP_030', 'VP_031',
       'VP_032'],
      dtype='object')

In [40]:
model_dict = {
    "scaler": {"StandardScaler": StandardScaler(), "MinMaxScaler": MinMaxScaler()},
    "reduce_dim": {"SelectKBest": SelectKBest(), "RFE": RFE(SVR(kernel="linear")), "DummyFeatureSelector": FunctionTransformer()},
    #"reduce_dim": {"SelectKBest": SelectKBest()},
    "clf": {
        #"LinearRegressor": LinearRegression(),
        #"ElasticNetRegressor": ElasticNet(),
        #"RidgeRegressor": Ridge(),
        "SVR": SVR(),
        #"KNeighborsRegressor": KNeighborsRegressor(),
        #"RandomForestRegressor": RandomForestRegressor(n_jobs=10),
    },
}

In [41]:
params_dict = {
    "StandardScaler": None,
    "MinMaxScaler": None,
    "SelectKBest": {
        "score_func": [f_regression, mutual_info_regression],
        "k": [2, 4, 6, 8, 10, "all"],
        },
    "SVR": {
        'kernel': ["linear"],
        'C': [0.01, 0.1, 1, 10, 100], 
        'epsilon': [0.001, 0.01, 0.1, 1],
        },
    "RFE": {
        "n_features_to_select": [0.4, 0.5, 0.6],
        "step": [1,2,3],
        },
    "DummyFeatureSelector": None, 
    #"LinearRegressor": {
    #},
    #"ElasticNetRegressor": {
    #    "alpha": [0.01, 0.1, 1, 10, 100],
    #    "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
    #},
    #"RidgeRegressor": {
    #    "alpha": [0.01, 0.1, 1, 10, 100],
    #}
    #"KNeighborsRegressor": {
    #    "n_neighbors": [8,9,10,11,12,13,14],
    #    "weights": ["uniform", "distance"],
    #    "p": [1,2],
    #    },
    #"RandomForestRegressor": {
    #    "n_estimators": [200, 300],
    #    "min_samples_split": [2, 5, 10, 20, 30],
    #    "min_samples_leaf": [1, 2, 4, 10, 20],
    #    "max_depth": [None, 40, 60, 80, 100],
    #    #"max_features": ["sqrt", "log2", None],
    #},
}

In [42]:
pipeline_permuter = SklearnPipelinePermuter(
    model_dict=model_dict, param_dict=params_dict
)

In [43]:
outer_cv = GroupKFold(n_splits=5)
inner_cv = GroupKFold(n_splits=5)

pipeline_permuter.fit(X=X, y=y, outer_cv=outer_cv, inner_cv=inner_cv, scoring="neg_mean_absolute_error", groups=groups)

Pipeline Combinations:   0%|          | 0/6 [00:00<?, ?it/s]

### Running hyperparameter search for pipeline: (('scaler', 'StandardScaler'), ('reduce_dim', 'SelectKBest'), ('clf', 'SVR')) with 1 parameter grid(s):
Parameter grid #0 ({'search_method': 'grid'}): {'reduce_dim__score_func': [<function f_regression at 0x000002DA043E5760>, <function mutual_info_regression at 0x000002DA043D3600>], 'reduce_dim__k': [2, 4, 6, 8, 10, 'all'], 'clf__kernel': ['linear'], 'clf__C': [0.01, 0.1, 1, 10, 100], 'clf__epsilon': [0.001, 0.01, 0.1, 1]}


Outer CV:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


...

C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


Fitting 5 folds for each of 180 candidates, totalling 900 fits


C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


Fitting 5 folds for each of 180 candidates, totalling 900 fits


KeyboardInterrupt: 

In [21]:
pipeline_permuter.metric_summary()

true_labels  \
pipeline_scaler pipeline_reduce_dim  pipeline_clf                                                             
StandardScaler  SelectKBest          LinearRegressor      [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     ElasticNetRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     RidgeRegressor       [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                RFE                  LinearRegressor      [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     ElasticNetRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     RidgeRegressor       [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                DummyFeatureSelector LinearRegressor      [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     ElasticNetRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     RidgeRegressor       [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
MinMaxScaler    SelectKBest          LinearRegressor      [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     ElasticNetRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     RidgeRegressor       [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                RFE                  LinearRegressor      [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     ElasticNetRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     RidgeRegressor       [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                DummyFeatureSelector LinearRegressor      [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     ElasticNetRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                     RidgeRegressor       [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   

                                                                                          true_labels_folds  \
pipeline_scaler pipeline_reduce_dim  pipeline_clf                                                             
StandardScaler  SelectKBest          LinearRegressor      [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     ElasticNetRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     RidgeRegressor       [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                RFE                  LinearRegressor      [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     ElasticNetRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     RidgeRegressor       [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                DummyFeatureSelector LinearRegressor      [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     ElasticNetRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     RidgeRegressor       [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
MinMaxScaler    SelectKBest          LinearRegressor      [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     ElasticNetRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     RidgeRegressor       [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                RFE                  LinearRegressor      [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     ElasticNetRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                                     RidgeRegressor       [[370.0, 346.0, 374.0, 316.0, 336.0, 322.

In [22]:
pipeline_permuter.best_hyperparameter_pipeline()

C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\sklearn_pipeline_permuter.py:680: FutureWarning: ['param_reduce_dim__k', 'param_reduce_dim__score_func', 'params'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  .agg(["mean", "std"])


,mean_test_neg_mean_absolute_error,param_reduce_dim__k,param_reduce_dim__score_func,params,rank_test_neg_mean_absolute_error,split0_test_neg_mean_absolute_error,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,split3_test_neg_mean_absolute_error,split4_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error
outer_fold,,,,,,,,,,,
0,-10.098984,4,<function mutual_info_regression at 0x000002DA...,"{'reduce_dim__k': 4, 'reduce_dim__score_func':...",1,-10.399348,-9.782195,-10.520335,-9.696486,-10.096556,0.325613
1,-10.507486,4,<function mutual_info_regression at 0x000002DA...,"{'reduce_dim__k': 4, 'reduce_dim__score_func':...",1,-11.732540,-10.625196,-10.964389,-9.685699,-9.529605,0.819141
2,-10.411076,4,<function mutual_info_regression at 0x000002DA...,"{'reduce_dim__k': 4, 'reduce_dim__score_func':...",1,-10.153486,-11.515515,-12.196229,-9.342831,-8.847322,1.269597
3,-10.023166,4,<function mutual_info_regression at 0x000002DA...,"{'reduce_dim__k': 4, 'reduce_dim__score_func':...",1,-9.009859,-10.419860,-11.371450,-9.563228,-9.751433,0.810852
4,-10.153753,4,<function mutual_info_regression at 0x000002DA...,"{'reduce_dim__k': 4, 'reduce_dim__score_func':...",1,-9.950579,-9.776165,-11.577767,-10.333368,-9.130884,0.811195


In [23]:
print(f"Save results: {save_results}")

Save results: True


In [24]:
if save_results:
    pipeline_permuter.to_pickle(models_path.joinpath("linear_models_b_point_rr.pkl"))